In [1]:
from DS_Planck_Unet import *
import numpy as np
import pandas as pd

In [2]:
def false_clusters(n, nside, clusters_dir, bigpixels, max_rad=1):
    import os
    import pandas as pd
    import numpy as np
    import healpy as hp
    from astropy.coordinates import SkyCoord
    from astropy import units as u
    
    files = next(os.walk(clusters_dir))[-1]
    clusters = []
    for file in files:
        clusters.append(pd.read_csv(os.path.join(clusters_dir, file)))
    clusters = pd.concat(clusters)
    
    no_clusters = set(np.arange(hp.nside2npix(nside)))
    sc = SkyCoord(ra=clusters['RA']*u.degree, dec=clusters['DEC']*u.degree, frame='icrs')
    vecs = hp.ang2vec(theta=sc.galactic.l.degree, phi=sc.galactic.b.degree, lonlat=True)
    for vec in vecs:
        no_clusters = no_clusters.difference(set(hp.query_disc(vec=vec, nside=nside, 
                            nest=True, radius=np.radians(max_rad), inclusive=False)))
        
    no_clusters = np.array(list(no_clusters))
    ang = hp.pix2ang(nside=nside, ipix=no_clusters, nest=True, lonlat=True)
    bp = hp.ang2pix(nside=2, theta=ang[0], phi=ang[1], lonlat=True, nest=True)
    no_clusters = no_clusters[np.in1d(bp, bigpixels)]
    pics, matrs = gen_batch(np.array(no_clusters), n, nside, 
                            pd.DataFrame({'RA' : [], 'DEC': []}))
    return pics, matrs

In [3]:
def clusters_in_pix(clusters, pix, nside):
    import pandas as pd
    import healpy as hp
    import numpy as np
    from DS_healpix_fragmentation import radec2pix
    
    df = pd.read_csv(clusters)
    cl_pix = radec2pix(df['RA'], df['DEC'], nside)
    df = df[cl_pix == pix]
    df.index = np.arange(df.shape[0])
    
    return df

In [4]:
def scan_pix(clusters, model, ipix, nside=2, depth=10, thr=0.8, min_dist=5/60, 
             step=64, size=64, n_false=None, search_nside=256):
    from DS_healpix_fragmentation import one_pixel_fragmentation, pix2radec, radec2pix
    from DS_Planck_Unet import draw_pic_with_mask
    from DS_detector import find_centers_on_ans
    from astropy.coordinates import SkyCoord
    from astropy import units as u
    import pandas as pd
    import numpy as np
    import healpy as hp
    from tensorflow.keras import backend as K
    from tqdm.notebook import tqdm
    
    big_matr = one_pixel_fragmentation(nside, ipix, depth)
    
    true_clusters = clusters_in_pix(clusters, ipix, nside)
    if n_false is None:
        n_false = len(true_clusters)
    
    pics, matrs = [], []
    
    for i in range(0, big_matr.shape[0] - step, step):
        for j in range(0, big_matr.shape[1], step):
            matr = big_matr[i:i+size,j:j+size]
            if matr.shape[0] == size and matr.shape[1] == size:
                pic = draw_pic(matr)
                pics.append(pic)
                matrs.append(matr)
    pics = np.array(pics)
    ans = model.predict(pics)
    ans = np.array(ans)
    found_clusters = find_centers_on_ans(ans, matrs, thr)
    all_found = len(found_clusters)
    theta, phi = hp.pix2ang(ipix=found_clusters, nest=True, nside=nside*2**depth,
                           lonlat=True)    
    sc_true = SkyCoord(ra=true_clusters['RA']*u.degree, 
                       dec=true_clusters['DEC']*u.degree, frame='icrs')
    sc_found = SkyCoord(l=theta*u.degree,
                       b=phi*u.degree, frame='galactic')
    
    idx, d2d, _ = sc_found.match_to_catalog_sky(sc_true)
    tp = np.count_nonzero(d2d.degree <= min_dist)
    fn = true_clusters.shape[0] - tp
    #----test false clusters----#
    pics, matrs = false_clusters(n_false, search_nside, '/home/rt2122/Data/clusters/',
                                [ipix])
    pics = np.array(pics)
    ans = model.predict(pics)
    ans = np.array(ans)
    found_clusters, tn = find_centers_on_ans(ans, matrs, thr, count_blanck=True)
    fp = len(found_clusters)
    
    
    
    res_table = pd.DataFrame({'tp': tp, 'fp': fp, 'tn': tn, 'fn': fn, 
                              'all_found' : all_found,
                             'min_dist' : d2d.degree.min(), 'pix2' : ipix}, index=[0])
    return res_table


In [5]:
model = unet_planck(weights='/home/rt2122/Models/model.01-0.10.h5')

In [6]:
res = scan_pix('/home/rt2122/Data/clusters/planck_z.csv', model, train_pix[0], thr=0.7)

/home/rt2122/L/git/data-segmentation-2/modules/DS_detector.py:47: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  centers.extend(list(matrs[i][[new_cen[:, 0], new_cen[:, 1]]]))


In [7]:
res1 = scan_pix('/home/rt2122/Data/clusters/planck_z.csv', model, test_pix[0], thr=0.7)

In [8]:
pd.concat([res, res1])

,tp,fp,tn,fn,all_found,min_dist,pix2
0,5,11,13,15,169,0.008416,0
0,17,49,19,25,280,0.007522,7
